In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('pracitce-364603-f0aa0101f81d.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#
date = 1 #当日は0、前日は1・・・
#date = 1 #当日は0、前日は1・・・
date2 = "08/16"
pmc = 11046012 # 店舗ID
SPREADSHEET_KEY = '16r8MyobAeuM_NeFxgkUeIrnHEpgbj78voQj6j-yKVRg' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 501 # スロットの1台目
wait = 0.02

user_id = "niwaniwa478@gmail.com"
user_pw = "niwaniwa28"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
path = "C:\\Users\\garden026\\WD\\" + str(wd) + "\\chromedriver.exe"

chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p50row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p50table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "50":
                    p50table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "50":
                        p50table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "549":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "549":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.5RAW'!F" + str(int(p50row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p50table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/16', '025765', '1', '24359.0', '-6319.0', '19', '9']
['08/16', '025765', '2', '25128.0', '7524.0', '44', '14']
['08/16', '025765', '3', '20000.0', '2408.0', '29', '8']
['08/16', '025765', '4', '24615.0', '9330.0', '46', '15']
['08/16', '025765', '5', '20769.0', '-2106.0', '23', '8']
['08/16', '025877', '6', '26410.0', '-10832.0', '14', '5']
['08/16', '025877', '7', '26154.0', '302.0', '31', '14']
['08/16', '025877', '8', '14615.0', '3010.0', '20', '5']
['08/16', '025877', '9', '22564.0', '904.0', '24', '9']
['08/16', '025877', '10', '27179.0', '-2707.0', '28', '13']
['08/16', '025834', '11', '32821.0', '15649.0', '26', '5']
['08/16', '025834', '12', '33333.0', '-16851.0', '6', '4']
['08/16', '025834', '13', '35385.0', '5719.0', '21', '3']
['08/16', '025834', '14', '37692.0', '-10231.0', '12', '4']
['08/16', '025834', '15', '22308.0', '-902.0', '11', '4']
['08/16', '025834', '16', '29487.0', '-3309.0', '12', '4']
['08/16', '025834', '17', '28718.0', '-3610.0', '11', '7']
['08/16', 

['08/16', '025579', '140', '42821.0', '904.0', '22', '7']
['08/16', '025895', '141', '40513.0', '1506.0', '27', '5']
['08/16', '025895', '142', '43077.0', '302.0', '27', '4']
['08/16', '025895', '143', '32564.0', '48751.0', '55', '9']
['08/16', '025895', '144', '40000.0', '-5416.0', '21', '6']
['08/16', '025895', '145', '43077.0', '54769.0', '65', '11']
['08/16', '025895', '146', '42564.0', '3311.0', '33', '10']
['08/16', '025895', '147', '40000.0', '15047.0', '36', '8']
['08/16', '025895', '148', '44615.0', '-26781.0', '10', '4']
['08/16', '025895', '149', '41795.0', '-29490.0', '7', '4']
['08/16', '025895', '150', '41026.0', '-25578.0', '11', '7']
['08/16', '025856', '151', '34872.0', '26181.0', '38', '6']
['08/16', '025856', '152', '14615.0', '14746.0', '18', '2']
['08/16', '025856', '153', '37179.0', '-10832.0', '15', '7']
['08/16', '025856', '154', '35385.0', '-20462.0', '7', '6']
['08/16', '025856', '155', '21282.0', '15348.0', '23', '5']
['08/16', '025856', '156', '33590.0', '-1

['08/16', '025758', '277', '35128.0', '-18055.0', '12', '7']
['08/16', '025775', '278', '21538.0', '25580.0', '27', '6']
['08/16', '025562', '279', '41795.0', '-7221.0', '26', '7']
['08/16', '025700', '280', '23333.0', '-15045.0', '7', '3']
['08/16', '025795', '281', '31538.0', '302.0', '32', '8']
['08/16', '025750', '282', '50256.0', '-21365.0', '26', '6']
['08/16', '025740', '283', '21026.0', '12941.0', '61', '2']
['08/16', '025696', '284', '25897.0', '17756.0', '42', '5']
['08/16', '025800', '285', '16154.0', '-8726.0', '6', '3']
['08/16', '025703', '286', '42051.0', '-902.0', '66', '10']
['08/16', '025625', '287', '38462.0', '-12939.0', '35', '6']
['08/16', '025853', '288', '20256.0', '-14143.0', '6', '2']
['08/16', '025437', '289', '27949.0', '-14444.0', '8', '4']
['08/16', '025550', '290', '30513.0', '-4814.0', '14', '3']
['08/16', '025554', '291', '34359.0', '-5115.0', '26', '6']
['08/16', '025801', '292', '37692.0', '-20462.0', '8', '4']
['08/16', '025796', '293', '23590.0', '-

['08/16', '025701', '416', '6667.0', '-4814.0', '1', '1']
['08/16', '025805', '417', '5385.0', '2709.0', '13', '2']
['08/16', '025837', '418', '6667.0', '12038.0', '9', '1']
['08/16', '025852', '419', '5385.0', '603.0', '10', '1']
['08/16', '025750', '420', '21538.0', '-6018.0', '36', '3']
['08/16', '025864', '421', '44872.0', '-12337.0', '34', '7']
['08/16', '025670', '422', '43846.0', '-23772.0', '14', '6']
['08/16', '025825', '423', '34103.0', '27385.0', '38', '6']
['08/16', '025838', '424', '53590.0', '-22268.0', '19', '6']
['08/16', '025826', '425', '35641.0', '3311.0', '23', '4']
['08/16', '025825', '426', '38462.0', '-29791.0', '4', '3']
['08/16', '025707', '427', '40000.0', '-20161.0', '14', '7']
['08/16', '025489', '428', '25385.0', '17154.0', '22', '2']
['08/16', '025553', '429', '52051.0', '19561.0', '65', '15']
['08/16', '025605', '430', '41026.0', '-9027.0', '29', '16']
['08/16', '025317', '431', '36923.0', '-8124.0', '30', '11']
['08/16', '025604', '432', '36410.0', '-361

['08/16', '120076', '533', '2111', '-93.0', '0', '0', '14']
['08/16', '120149', '534', '3876', '-1111.0', '0', '9', '37']
['08/16', '120121', '535', '1716', '-833.0', '0', '3', '18']
['08/16', '120069', '536', '1721', '324.0', '0', '3', '11']
['08/16', '120034', '537', '2786', '-370.0', '0', '3', '20']
['08/16', '120034', '538', '2829', '1065.0', '0', '5', '26']
['08/16', '120034', '539', '4235', '-3565.0', '0', '2', '16']
['08/16', '120034', '540', '792', '-787.0', '0', '0', '2']
['08/16', '120103', '561', '5741', '-3889.0', '17', '7', '23']
['08/16', '120103', '562', '5195', '-1481.0', '23', '6', '18']
['08/16', '120103', '563', '4927', '4398.0', '78', '2', '10']
['08/16', '120103', '564', '2152', '602.0', '23', '2', '4']
['08/16', '120103', '565', '6290', '2778.0', '49', '10', '23']
['08/16', '120138', '566', '0', '0', '0', '0', '--']
['08/16', '120138', '567', '1133', '-417.0', '0', '2', '3']
['08/16', '120138', '568', '1040', '-1250.0', '0', '1', '1']
['08/16', '120138', '569', '0

['08/16', '120043', '687', '6100', '2222.0', '31', '16', '--']
['08/16', '120065', '688', '1488', '-1111.0', '3', '2', '--']
['08/16', '120071', '689', '899', '139.0', '5', '1', '--']
['08/16', '120088', '690', '1555', '1435.0', '7', '3', '13']
['08/16', '120137', '691', '2530', '-278.0', '6', '9', '15']
['08/16', '120137', '692', '3593', '-1204.0', '7', '7', '14']
['08/16', '119962', '693', '689', '-324.0', '2', '0', '--']
['08/16', '119962', '694', '1835', '-1435.0', '1', '2', '--']
['08/16', '119962', '695', '1187', '-417.0', '3', '1', '--']
['08/16', '120026', '696', '6401', '185.0', '23', '16', '--']
['08/16', '120026', '697', '1290', '-463.0', '3', '3', '--']
['08/16', '120026', '698', '566', '-648.0', '0', '0', '--']
['08/16', '120026', '699', '1109', '-324.0', '3', '1', '--']
['08/16', '120026', '700', '1669', '-185.0', '6', '3', '--']
['08/16', '120073', '701', '5521', '3889.0', '0', '12', '53']
['08/16', '120073', '702', '838', '-880.0', '0', '2', '3']
['08/16', '120073', '70

{'spreadsheetId': '16r8MyobAeuM_NeFxgkUeIrnHEpgbj78voQj6j-yKVRg',
 'updates': {'spreadsheetId': '16r8MyobAeuM_NeFxgkUeIrnHEpgbj78voQj6j-yKVRg',
  'updatedRange': 'S5RAW!F742:M761',
  'updatedRows': 20,
  'updatedColumns': 8,
  'updatedCells': 160}}

In [32]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p50table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "50":
                    p50table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "50":
                        p50table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "549":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "549":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.5RAW'!F" + str(int(p50row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p50table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['07/20', '025765', '1', '20256.0', '3612.0', '28', '12']
['07/20', '025765', '2', '18462.0', '4214.0', '27', '7']
['07/20', '025765', '3', '22821.0', '5117.0', '37', '13']
['07/20', '025765', '4', '8718.0', '-902.0', '7', '3']
['07/20', '025765', '5', '7949.0', '-1805.0', '6', '3']
['07/20', '025877', '6', '41538.0', '12038.0', '63', '12']
['07/20', '025877', '7', '16667.0', '-4814.0', '11', '6']
['07/20', '025877', '8', '43846.0', '-5115.0', '40', '18']
['07/20', '025877', '9', '25128.0', '4515.0', '36', '13']
['07/20', '025877', '10', '27692.0', '-6018.0', '23', '14']
['07/20', '025834', '11', '25897.0', '4515.0', '15', '4']
['07/20', '025834', '12', '37692.0', '-15346.0', '9', '4']
['07/20', '025834', '13', '29487.0', '-24073.0', '0', '0']
['07/20', '025834', '14', '40769.0', '23473.0', '34', '11']
['07/20', '025834', '15', '15641.0', '-6319.0', '4', '1']
['07/20', '025834', '16', '22564.0', '-6619.0', '7', '3']
['07/20', '025834', '17', '33590.0', '8126.0', '21', '3']
['07/20', '0

['07/20', '025856', '142', '5128.0', '10232.0', '10', '1']
['07/20', '025856', '143', '12308.0', '-2106.0', '6', '2']
['07/20', '025856', '144', '16923.0', '-3309.0', '8', '3']
['07/20', '025856', '145', '11026.0', '-4212.0', '4', '3']
['07/20', '025856', '146', '8974.0', '11737.0', '13', '1']
['07/20', '025856', '147', '13077.0', '9330.0', '14', '2']
['07/20', '025867', '148', '4872.0', '302.0', '3', '1']
['07/20', '025867', '149', '18718.0', '-15647.0', '1', '1']
['07/20', '025867', '150', '7949.0', '-7221.0', '0', '0']
['07/20', '025867', '151', '6923.0', '4816.0', '8', '2']
['07/20', '025867', '152', '16410.0', '-8425.0', '5', '1']
['07/20', '025864', '153', '7949.0', '-6319.0', '1', '1']
['07/20', '025864', '154', '6923.0', '-3610.0', '3', '1']
['07/20', '025864', '155', '3590.0', '6922.0', '15', '1']
['07/20', '025864', '156', '2051.0', '5719.0', '9', '1']
['07/20', '025864', '157', '10000.0', '904.0', '12', '2']
['07/20', '025864', '158', '513.0', '-902.0', '0', '0']
['07/20', '

['07/20', '025740', '283', '12564.0', '-9629.0', '1', '1']
['07/20', '025696', '284', '12308.0', '-7221.0', '5', '2']
['07/20', '025878', '285', '25897.0', '-9328.0', '25', '4']
['07/20', '025679', '286', '11026.0', '9931.0', '56', '4']
['07/20', '025625', '287', '16667.0', '-3008.0', '20', '4']
['07/20', '025853', '288', '12821.0', '-10531.0', '2', '2']
['07/20', '025437', '289', '5641.0', '-5115.0', '0', '0']
['07/20', '025550', '290', '2821.0', '3010.0', '4', '1']
['07/20', '025554', '291', '9231.0', '-1203.0', '9', '2']
['07/20', '025801', '292', '23846.0', '-1203.0', '15', '4']
['07/20', '025796', '293', '17692.0', '5418.0', '17', '3']
['07/20', '025483', '294', '4872.0', '-2406.0', '2', '1']
['07/20', '025578', '295', '18205.0', '1806.0', '12', '3']
['07/20', '025779', '296', '6154.0', '-1805.0', '4', '1']
['07/20', '025838', '297', '17436.0', '1205.0', '15', '7']
['07/20', '025838', '298', '2564.0', '-1203.0', '1', '1']
['07/20', '025707', '299', '29231.0', '-20763.0', '5', '5']

['07/20', '025825', '426', '13590.0', '11135.0', '16', '3']
['07/20', '025707', '427', '17179.0', '-13541.0', '4', '4']
['07/20', '025489', '428', '15385.0', '-10531.0', '1', '1']
['07/20', '025553', '429', '18718.0', '2408.0', '17', '4']
['07/20', '025605', '430', '43590.0', '3612.0', '53', '19']
['07/20', '025317', '431', '16154.0', '-5416.0', '11', '7']
['07/20', '025604', '432', '27692.0', '2709.0', '28', '10']
['07/20', '025655', '433', '13846.0', '2107.0', '37', '10']
['07/20', '025689', '434', '26923.0', '-1504.0', '32', '12']
['07/20', '025605', '435', '29487.0', '904.0', '36', '14']
['07/20', '025369', '436', '26154.0', '-2106.0', '57', '19']
['07/20', '025533', '437', '13077.0', '6621.0', '31', '6']
['07/20', '025726', '438', '9487.0', '-2707.0', '7', '3']
['07/20', '025655', '439', '23846.0', '603.0', '52', '12']
['07/20', '025605', '440', '39487.0', '1806.0', '46', '18']
['07/20', '025722', '441', '37692.0', '-4212.0', '48', '15']
['07/20', '025797', '442', '23333.0', '-722

['07/20', '120138', '566', '1588', '-1157.0', '0', '2', '4']
['07/20', '120138', '567', '2404', '-139.0', '0', '4', '5']
['07/20', '120138', '568', '1730', '-648.0', '0', '2', '4']
['07/20', '120138', '569', '2249', '-880.0', '0', '4', '6']
['07/20', '120130', '570', '5303', '-4954.0', '0', '8', '11']
['07/20', '120130', '571', '2602', '1435.0', '0', '4', '13']
['07/20', '120130', '572', '2911', '-2824.0', '0', '3', '8']
['07/20', '120130', '573', '3540', '-4167.0', '0', '5', '8']
['07/20', '120130', '574', '3919', '-1019.0', '0', '8', '19']
['07/20', '120122', '575', '2535', '-2037.0', '14', '7', '--']
['07/20', '120122', '576', '3011', '-1157.0', '22', '7', '--']
['07/20', '120122', '577', '5663', '463.0', '63', '25', '--']
['07/20', '120122', '578', '826', '-417.0', '6', '3', '--']
['07/20', '120122', '579', '1697', '4676.0', '48', '3', '--']
['07/20', '120122', '580', '2298', '9398.0', '96', '2', '--']
['07/20', '120010', '581', '2752', '926.0', '12', '13', '--']
['07/20', '120010'

['07/20', '120073', '702', '3583', '1389.0', '0', '12', '33']
['07/20', '120073', '703', '2807', '-787.0', '0', '8', '17']
['07/20', '120073', '704', '3285', '926.0', '0', '7', '24']
['07/20', '120073', '705', '1298', '-93.0', '0', '4', '9']
['07/20', '120073', '706', '5526', '1111.0', '0', '14', '41']
['07/20', '120073', '707', '3609', '-1435.0', '0', '8', '19']
['07/20', '120073', '708', '1482', '-1296.0', '0', '6', '4']
['07/20', '120093', '709', '2692', '-93.0', '21', '0', '21']
['07/20', '120093', '710', '1168', '-139.0', '8', '0', '8']
['07/20', '120093', '711', '263', '-278.0', '0', '0', '--']
['07/20', '120093', '712', '3116', '3426.0', '38', '0', '38']
['07/20', '120077', '713', '1256', '-972.0', '0', '2', '4']
['07/20', '120077', '714', '5186', '-556.0', '0', '13', '36']
['07/20', '120077', '715', '1884', '231.0', '0', '7', '13']
['07/20', '120126', '716', '3845', '1389.0', '0', '10', '27']
['07/20', '120126', '717', '3159', '-370.0', '0', '6', '17']
['07/20', '120126', '718'

{'spreadsheetId': '16r8MyobAeuM_NeFxgkUeIrnHEpgbj78voQj6j-yKVRg',
 'updates': {'spreadsheetId': '16r8MyobAeuM_NeFxgkUeIrnHEpgbj78voQj6j-yKVRg',
  'updatedRange': 'S5RAW!F242:M261',
  'updatedRows': 20,
  'updatedColumns': 8,
  'updatedCells': 160}}